# Installing NewsAPI
This part will be used to collect real time news datasets for the our project

Please refer to here how to install the https://github.com/mattlisiv/newsapi-python

Also, please make sure the change the necessary paths in the code. Because I used colab for implementing.
For example, you will need to change the paths inside of "event-registry-python/setup.py"


In [ ]:
!pip install newsapi-python


In [ ]:
API_KEY = ''

In [ ]:
!pip install eventregistry

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eventregistry: filename=eventregistry-9.1-py3-none-any.whl size=67625 sha256=ce8add95599c3cea0ed7f4e4f72194e45fccc9834ae664e00d36982da262860d
  Stored in directory: /root/.cache/pip/wheels/43/b1/a3/96973dbeb71bb960bd053bfc7113194a3c35859407e20c907f
Successfully built eventregistry


In [ ]:
!git clone https://github.com/EventRegistry/event-registry-python.git

Cloning into 'event-registry-python'...
remote: Enumerating objects: 1475, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (122/122), done.
remote: Total 1475 (delta 97), reused 97 (delta 40), pack-reused 1306 (from 1)
Receiving objects: 100% (1475/1475), 646.63 KiB | 2.46 MiB/s, done.
Resolving deltas: 100% (1042/1042), done.


In [ ]:
!python /content/event-registry-python/setup.py install

running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https

In [ ]:
from newsapi import NewsApiClient

In [ ]:
api = NewsApiClient(api_key=API_KEY)

In [ ]:
api.get_top_headlines(sources='bbc-news')

{'status': 'ok',
 'totalResults': 10,
 'articles': [{'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': '"Uncommitted\' anger over Gaza threatens Harris in Michigan',
   'description': 'Arab-American voters in swing states like Michigan could decide who wins the US presidential election',
   'url': 'https://www.bbc.co.uk/news/articles/c8xejx07exro',
   'urlToImage': 'https://ichef.bbci.co.uk/news/1024/branded_news/6690/live/342a01c0-7772-11ef-8c1a-df523ba43a9a.jpg',
   'publishedAt': '2024-09-21T20:37:11.1718744Z',
   'content': 'In February\'s Democratic primary in Michigan, part of the process of choosing the party\'s presidential candidate, over 100,000 people in this community declared themselves "uncommitted" in protest at… [+1547 chars]'},
  {'source': {'id': 'bbc-news', 'name': 'BBC News'},
   'author': 'BBC News',
   'title': 'Donald Trump turns down second debate with Kamala Harris',
   'description': 'Kamala Harris had "gladly" accepted CNN

In [ ]:
api.get_everything(q='apple')

{'status': 'ok',
 'totalResults': 6970,
 'articles': [{'source': {'id': 'wired', 'name': 'Wired'},
   'author': 'Joel Khalili',
   'title': 'The World’s Biggest Bitcoin Mine Is Rattling This Texas Oil Town',
   'description': 'A cash-strapped city in rural Texas will soon be home to the world’s largest bitcoin mine. Local protesters are “raising hell.”',
   'url': 'https://www.wired.com/story/the-worlds-biggest-bitcoin-mine-is-rattling-this-texas-oil-town/',
   'urlToImage': 'https://media.wired.com/photos/66c5ecc5724cee849e3104da/191:100/w_1280,c_limit/WIRED_BTC_EC_B-Elena-Chudoba.jpg',
   'publishedAt': '2024-09-11T10:00:00Z',
   'content': 'The previous October, Sawicky organized a weeklong protest alongside environmental activist group Greenpeace and brandished various anti-bitcoin signs at anyone who entered the Riot facility. Only a … [+3641 chars]'},
  {'source': {'id': None, 'name': 'Yahoo Entertainment'},
   'author': 'Jeremy Gan',
   'title': 'Proton Drive now has a tier for 

In [ ]:
import requests
import sqlite3
import csv
from datetime import datetime

# Function to fetch news articles
def get_news(api_key, query):
    url = f'https://newsapi.org/v2/everything?q={query}&apiKey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()['articles']
    else:
        print("Failed to retrieve news")
        return []



# Function to save articles to the database
def save_to_db(articles):
    conn = sqlite3.connect('news.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS news (
            id INTEGER PRIMARY KEY,
            title TEXT,
            description TEXT,
            content TEXT,
            url TEXT,
            published_at TEXT
        )
    ''')

    for article in articles:
        c.execute('''
            INSERT INTO news (title, description, content, url, published_at)
            VALUES (?, ?, ?, ?, ?)
        ''', (article['title'], article['description'], article['content'], article['url'], article['publishedAt']))

    conn.commit()
    conn.close()

# Function to save articles as a CSV file
def save_to_csv(articles, filename='news.csv'):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'Description', 'Content', 'URL', 'Published At'])

        for article in articles:
            writer.writerow([
                article['title'],
                article['description'],
                article['content'],
                article['url'],
                article['publishedAt']
            ])

# Main function
def main():
    api_key = API_KEY
    queries = ['apple', 'bitcoin', 'finance', 'stock', 'election']

    all_articles = []
    for query in queries:
        articles = get_news(api_key, query)
        all_articles.extend(articles)

    save_to_db(all_articles)
    save_to_csv(all_articles)

if __name__ == '__main__':
    main()


In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
stocks = ["AAPL", "MSFT", "AMZN", "GOOG", "GOOGL", "TSLA", "NVDA", "META", "UNH", "JNJ",
          "V", "JPM", "PG", "HD", "MA", "BAC", "XOM", "CVX", "LLY", "PFE"]

data = yf.download(stocks, period="1mo")

[*********************100%***********************]  20 of 20 completed


In [ ]:
df = pd.DataFrame(data)
df

Price                       Adj Close                                     \
Ticker                           AAPL        AMZN        BAC         CVX   
Date                                                                       
2024-08-21 00:00:00+00:00  226.399994  180.110001  38.499001  145.320007   
2024-08-22 00:00:00+00:00  224.529999  176.130005  38.985828  145.729996   
2024-08-23 00:00:00+00:00  226.839996  177.039993  39.512398  147.619995   
2024-08-26 00:00:00+00:00  227.179993  175.500000  39.661423  148.500000   
2024-08-27 00:00:00+00:00  228.029999  173.119995  39.413040  146.949997   
2024-08-28 00:00:00+00:00  226.490005  170.800003  39.691231  146.000000   
2024-08-29 00:00:00+00:00  229.789993  172.119995  39.909801  147.419998   
2024-08-30 00:00:00+00:00  229.000000  178.500000  40.486046  147.949997   
2024-09-03 00:00:00+00:00  222.770004  176.250000  40.436371  144.660004   
2024-09-04 00:00:00+00:00  220.850006  173.330002  40.237667  142.070007   
2024-09-05 00:00:00+00:00  222.380005  177.889999  39.879997  140.929993   
2024-09-06 00:00:00+00:00  220.820007  171.389999  38.759998  138.559998   
2024-09-09 00:00:00+00:00  220.910004  175.399994  39.470001  140.300003   
2024-09-10 00:00:00+00:00  220.110001  179.550003  39.279999  138.229996   
2024-09-11 00:00:00+00:00  222.660004  184.520004  39.000000  137.880005   
2024-09-12 00:00:00+00:00  222.770004  187.000000  38.779999  139.210007   
2024-09-13 00:00:00+00:00  222.500000  186.490005  38.650002  140.610001   
2024-09-16 00:00:00+00:00  216.320007  184.889999  39.099998  142.089996   
2024-09-17 00:00:00+00:00  216.789993  186.880005  39.549999  143.410004   
2024-09-18 00:00:00+00:00  220.690002  186.429993  39.619999  143.960007   
2024-09-19 00:00:00+00:00  228.869995  189.869995  40.869999  144.990005   
2024-09-20 00:00:00+00:00  228.199997  191.600006  40.270000  145.679993   

Price                                                                      \
Ticker                           GOOG       GOOGL          HD         JNJ   
Date                                                                        
2024-08-21 00:00:00+00:00  167.409637  165.630219  368.223450  160.213943   
2024-08-22 00:00:00+00:00  165.272446  163.582932  363.154236  161.127029   
2024-08-23 00:00:00+00:00  167.209885  165.400513  373.322479  162.893616   
2024-08-26 00:00:00+00:00  167.709229  165.939804  371.851410  163.369995   
2024-08-27 00:00:00+00:00  166.161270  164.461761  370.927032  162.949997   
2024-08-28 00:00:00+00:00  164.283737  162.634201  370.440002  163.919998   
2024-08-29 00:00:00+00:00  163.185181  161.565613  367.059998  164.229996   
2024-08-30 00:00:00+00:00  164.892944  163.163498  368.500000  165.860001   
2024-09-03 00:00:00+00:00  158.401489  157.151474  364.739990  167.160004   
2024-09-04 00:00:00+00:00  157.602539  156.242676  364.670013  167.360001   
2024-09-05 00:00:00+00:00  158.391510  157.031631  361.850006  164.990005   
2024-09-06 00:00:00+00:00  151.930008  150.720001  360.049988  164.380005   
2024-09-09 00:00:00+00:00  149.539993  148.710007  365.519989  166.610001   
2024-09-10 00:00:00+00:00  150.009995  148.660004  370.869995  167.380005   
2024-09-11 00:00:00+00:00  152.149994  151.160004  370.489990  164.820007   
2024-09-12 00:00:00+00:00  155.539993  154.690002  374.429993  164.639999   
2024-09-13 00:00:00+00:00  158.369995  157.460007  379.989990  165.520004   
2024-09-16 00:00:00+00:00  158.990005  158.059998  382.010010  166.990005   
2024-09-17 00:00:00+00:00  160.279999  159.320007  383.239990  167.070007   
2024-09-18 00:00:00+00:00  160.809998  159.809998  384.010010  166.149994   
2024-09-19 00:00:00+00:00  163.240005  162.139999  390.339996  164.820007   
2024-09-20 00:00:00+00:00  164.639999  163.589996  389.859985  164.160004   

Price                                              ...   Volume            \
Ticker                            JPM         LLY  ...       MA      META   
Date               

In [ ]:
def save_to_db(df):
    conn = sqlite3.connect('news.db')
    c = conn.cursor()
    c.execute('''
        CREATE TABLE IF NOT EXISTS stock (
            id INTEGER PRIMARY KEY,
            date DATE,
            price FLOAT
        )
    ''')

    cursor = conn.cursor()

    for index, row in df.iterrows():  # Using iterrows to access row data
        cursor.execute('''
            INSERT INTO stock (id, date, price)
            VALUES (?, ?, ?)
        ''', (row[0], row[1]))  # Accessing as a DataFrame row

    conn.commit()
    conn.close()


In [ ]:
save_to_db(df)

<ipython-input-89-07631f826302>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ''', (row[0], row[1]))  # Accessing as a DataFrame row


ProgrammingError: Incorrect number of bindings supplied. The current statement uses 3, and there are 2 supplied.